In [1]:
# Imports
import pandas as pd
import geopandas as gpd
import datetime
from datetime import date
import numpy as np

In [53]:
# Import districts
shp2_Kenya = gpd.read_file("git/Locust-Covid19/my_package/data/input/gadm36_KEN_2.shp")
shp2_Ethiopia = gpd.read_file("git/Locust-Covid19/my_package/data/input/gadm36_ETH_2.shp")
shp2_Somalia = gpd.read_file("git/Locust-Covid19/my_package/data/input/gadm36_SOM_2.shp")
shp2_Uganda = gpd.read_file("git/Locust-Covid19/my_package/data/input/gadm36_UGA_2.shp")

district_level = [shp2_Kenya, shp2_Ethiopia, shp2_Somalia, shp2_Uganda]
gdf_districts = gpd.GeoDataFrame(pd.concat(district_level, ignore_index=True))
gdf_districts.crs = {"init": "epsg:4326"}

In [54]:
gdf_districts.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,KEN,Kenya,KEN.1_1,Baringo,None,KEN.1.1_1,805,None,None,Constituency,Constituency,162,None,"POLYGON ((35.87727 -0.02973, 35.87699 -0.02947..."
1,KEN,Kenya,KEN.1_1,Baringo,None,KEN.1.2_1,Baringo Central,None,None,Constituency,Constituency,159,None,"POLYGON ((35.80651 0.31642, 35.80780 0.31627, ..."
2,KEN,Kenya,KEN.1_1,Baringo,None,KEN.1.3_1,Baringo North,None,None,Constituency,Constituency,158,None,"POLYGON ((35.81394 0.60442, 35.81377 0.60363, ..."
3,KEN,Kenya,KEN.1_1,Baringo,None,KEN.1.4_1,Baringo South,None,None,Constituency,Constituency,160,None,"POLYGON ((36.25757 0.38328, 36.25766 0.38242, ..."
4,KEN,Kenya,KEN.1_1,Baringo,None,KEN.1.5_1,Eldama Ravine,None,None,Constituency,Constituency,162,None,"POLYGON ((35.84734 -0.07654, 35.84637 -0.07804..."


In [55]:
# Import cropland vector
crops = gpd.read_file("git/Locust-Covid19/my_package/data/input/crops/Crops_vectorized.shp")
print(crops.head())

   fid  Crops                                           geometry
0  1.0      0  POLYGON ((37.77539 14.67733, 37.78475 14.67696...
1  2.0      0  POLYGON ((37.79349 14.66769, 37.80286 14.66732...
2  3.0      1  POLYGON ((37.83906 14.64804, 37.84842 14.64768...
3  4.0      1  POLYGON ((37.72551 14.63462, 37.73487 14.63425...
4  5.0      1  POLYGON ((37.88462 14.62839, 37.89398 14.62803...


In [56]:
print(crops.columns)

Index(['fid', 'Crops', 'geometry'], dtype='object')


In [57]:
crops = crops[crops['Crops'] == 1]
crops.Crops = 12
crops.head()

,fid,Crops,geometry
2,3.0,12,"POLYGON ((37.83906 14.64804, 37.84842 14.64768..."
3,4.0,12,"POLYGON ((37.72551 14.63462, 37.73487 14.63425..."
4,5.0,12,"POLYGON ((37.88462 14.62839, 37.89398 14.62803..."
5,6.0,12,"POLYGON ((37.90335 14.62766, 37.91271 14.62729..."
6,7.0,12,"POLYGON ((37.76984 14.59717, 37.77920 14.59680..."


In [58]:
crops.shape

(15282, 3)

In [59]:
gdf_districts_filtered = gdf_districts[['GID_2', 'geometry']]
crops_district = gpd.overlay(crops, gdf_districts_filtered, how='intersection')
crops_district.head()

,fid,Crops,GID_2,geometry
0,3.0,12,ETH.11.5_1,"POLYGON ((37.83906 14.64804, 37.84842 14.64768..."
1,4.0,12,ETH.11.5_1,"POLYGON ((37.73375 14.63430, 37.73487 14.63425..."
2,5.0,12,ETH.11.5_1,"POLYGON ((37.88462 14.62839, 37.89398 14.62803..."
3,6.0,12,ETH.11.5_1,"POLYGON ((37.90335 14.62766, 37.91271 14.62729..."
4,7.0,12,ETH.11.5_1,"POLYGON ((37.76984 14.59717, 37.77920 14.59680..."


In [60]:
crops_district['area_inter'] = crops_district.geometry.area
crops_district.head()

,fid,Crops,GID_2,geometry,area_inter
0,3.0,12,ETH.11.5_1,"POLYGON ((37.83906 14.64804, 37.84842 14.64768...",0.000084
1,4.0,12,ETH.11.5_1,"POLYGON ((37.73375 14.63430, 37.73487 14.63425...",0.000044
2,5.0,12,ETH.11.5_1,"POLYGON ((37.88462 14.62839, 37.89398 14.62803...",0.000084
3,6.0,12,ETH.11.5_1,"POLYGON ((37.90335 14.62766, 37.91271 14.62729...",0.000501
4,7.0,12,ETH.11.5_1,"POLYGON ((37.76984 14.59717, 37.77920 14.59680...",0.000084


In [61]:
crops_district['measureID'] = 27
crops_district['factID'] = 'CROP_'
crops_district['factID'] = crops_district['factID'] + crops_district['fid'].astype(str)
crops_district['year'] = 2015
crops_district['date'] = pd.to_datetime([f'{y}-01-01' for y in crops_district.year])
crops_district['locationID'] = crops_district['GID_2']
crops_district['value'] = crops_district['area_inter']

crops_district.head()

,fid,Crops,GID_2,geometry,area_inter,measureID,factID,year,date,locationID,value
0,3.0,12,ETH.11.5_1,"POLYGON ((37.83906 14.64804, 37.84842 14.64768...",0.000084,27,CROP_3.0,2015,2015-01-01,ETH.11.5_1,0.000084
1,4.0,12,ETH.11.5_1,"POLYGON ((37.73375 14.63430, 37.73487 14.63425...",0.000044,27,CROP_4.0,2015,2015-01-01,ETH.11.5_1,0.000044
2,5.0,12,ETH.11.5_1,"POLYGON ((37.88462 14.62839, 37.89398 14.62803...",0.000084,27,CROP_5.0,2015,2015-01-01,ETH.11.5_1,0.000084
3,6.0,12,ETH.11.5_1,"POLYGON ((37.90335 14.62766, 37.91271 14.62729...",0.000501,27,CROP_6.0,2015,2015-01-01,ETH.11.5_1,0.000501
4,7.0,12,ETH.11.5_1,"POLYGON ((37.76984 14.59717, 37.77920 14.59680...",0.000084,27,CROP_7.0,2015,2015-01-01,ETH.11.5_1,0.000084


In [62]:
dates = pd.read_csv("git/Locust-Covid19/my_package/data/input/date_23_06-2020.csv")
dates['date'] = pd.to_datetime(dates['date'])
dates.head()

,dateID,date
0,20000101,2000-01-01
1,20000102,2000-02-01
2,20000103,2000-03-01
3,20000104,2000-04-01
4,20000105,2000-05-01


In [63]:
crops_district = crops_district.merge(dates, on='date', how='left')
crops_district = crops_district[['factID', 'measureID', 'dateID', 'locationID', 'value']]
crops_district.head()

,factID,measureID,dateID,locationID,value
0,CROP_3.0,27,20150101,ETH.11.5_1,0.000084
1,CROP_4.0,27,20150101,ETH.11.5_1,0.000044
2,CROP_5.0,27,20150101,ETH.11.5_1,0.000084
3,CROP_6.0,27,20150101,ETH.11.5_1,0.000501
4,CROP_7.0,27,20150101,ETH.11.5_1,0.000084


In [64]:
# Export to parquet
crops_district.to_parquet('Cropland_20200708.parquet',
              compression='uncompressed', index = False)

In [65]:
# Export to csv
crops_district.to_csv('Cropland_20200708.csv', sep='|', encoding='utf-8', index = False)